Read the data and collect all the necessary information 

In [16]:
import pandas as pd  
import re  

In [17]:
def analyze_columns(file_path):  
    try:  
        df = pd.read_csv(file_path, nrows=0)  
        columns = df.columns.tolist()  
        
        # Initialize dictionaries to count occurrences  
        metric_counts = {}  
        industry_counts = {}  
        metric_pattern = r'\((.*?)\)'  
        
        for col in columns:   
            if col == 'Title':  
                continue  
                
            metric_match = re.search(metric_pattern, col)  
            if metric_match:  
                metric_raw = metric_match.group(1)  
                metric = ' '.join(metric_raw.strip().split()).lower()  
                metric_counts[metric] = metric_counts.get(metric, 0) + 1  
                
                industry_match = re.search(r'^(.*?)\s*\(', col)  
                if industry_match:  
                    industry = industry_match.group(1).strip()  
                    # Add to industry counts if not already counted  
                    if industry not in industry_counts:  
                        industry_counts[industry] = 1  

        total_metrics = sum(metric_counts.values())  
        sorted_metrics = sorted(  
            metric_counts.items(), key=lambda item: item[1], reverse=True  
        )  
        
        sorted_industries = sorted(industry_counts.keys())  

        print("\nMetrics Analysis:")  
        print(f"There are {len(sorted_metrics)} different types of metrics:")  
        for i, (metric, count) in enumerate(sorted_metrics, 1):  
            percentage = (count / total_metrics) * 100  
            display_metric = ' '.join(word.capitalize() for word in metric.split())  
            print(f"{i}. {display_metric}")  
            print(f"   Occurrence count: {count} ({percentage:.1f}%)")  

        print("\nIndustry sector analysis:")  
        print(f"There are {len(sorted_industries)} different industry sectors:")  
        for i, industry in enumerate(sorted_industries, 1):  
            print(f"{i}. {industry}")  

    except Exception as e:  
        print(f"An error occurred: {e}")  
        import traceback  
        traceback.print_exc()  

# File path  
file_path = 'mgdp.csv'  
analyze_columns(file_path)  


Metrics Analysis:
There are 9 different types of metrics:
1. Index 1dp
   Occurrence count: 23 (11.1%)
2. Period On Period Growth
   Occurrence count: 23 (11.1%)
3. Period On Period 1 Year Ago Growth
   Occurrence count: 23 (11.1%)
4. 3 Month On 3 Month Growth
   Occurrence count: 23 (11.1%)
5. 3m On 3m 1 Year Ago Growth
   Occurrence count: 23 (11.1%)
6. Period On Period Contribution
   Occurrence count: 23 (11.1%)
7. Period On Period 1 Year Ago Contribution
   Occurrence count: 23 (11.1%)
8. 3 Month On 3 Month Contribution
   Occurrence count: 23 (11.1%)
9. 3m On 3m 1 Year Ago Contribution
   Occurrence count: 23 (11.1%)

Industry sector analysis:
There are 23 different industry sectors:
1. Accommodation and food service activities
2. Activities of housholds as employers;Undiff goods & services
3. Administrative and Support Service Activities
4. Agriculture, Forestry and Fishing
5. Arts, Entertainment and Recreation
6. Construction
7. Education
8. Electricity, gas, steam and air con

Data clean (delete all ':CVM SA')

In [19]:
def clean_column_names(input_file, output_file='mgdp1.csv'):  
    """  
    Clean column names by removing ':CVM SA' and other unnecessary parts  
    while maintaining the "Industry (Metric)" format.  
    """  
    try:  
        # Read the CSV file  
        df = pd.read_csv(input_file)  
        
        # Get current column names  
        old_columns = df.columns.tolist()  
        
        # Clean column names by removing unnecessary parts  
        new_columns = []  
        
        for col in old_columns:  
            # Remove the ':CVM SA' suffix from column names  
            cleaned_col = re.sub(r'\s*:CVM SA\s*$', '', col)  
            
            # Add the cleaned column name to our list  
            new_columns.append(cleaned_col)  
        
        # Rename the dataframe columns  
        df.columns = new_columns  
        
        # Save the cleaned dataframe to a new CSV file  
        df.to_csv(output_file, index=False)  
        
        print(f"Successfully cleaned column names and saved to {output_file}")  
        print("\nOriginal vs. Cleaned Column Names (first 5):")  
        for i in range(min(5, len(old_columns))):  
            print(f"Original: {old_columns[i]}")  
            print(f"Cleaned:  {new_columns[i]}")  
            print("")  
        
        return df  
    
    except Exception as e:  
        print(f"Error cleaning column names: {e}")  
        import traceback  
        traceback.print_exc()  
        return None  

# Usage  
input_file = "mgdp.csv"  # Replace with your actual input file path  
output_file = "mgdp1.csv"  
cleaned_df = clean_column_names(input_file, output_file)  

Successfully cleaned column names and saved to mgdp1.csv

Original vs. Cleaned Column Names (first 5):
Original: Title
Cleaned:  Title

Original: Gross Value Added - Monthly (Index 1dp) :CVM SA
Cleaned:  Gross Value Added - Monthly (Index 1dp)

Original: Agriculture, Forestry and Fishing (Index 1dp) :CVM SA
Cleaned:  Agriculture, Forestry and Fishing (Index 1dp)

Original: Production Industries - Total (Index 1dp) :CVM SA
Cleaned:  Production Industries - Total (Index 1dp)

Original: Mining and Quarrying (Index 1dp) :CVM SA
Cleaned:  Mining and Quarrying (Index 1dp)



Data reclass
Date, Industry1 (metric 1), Industry1 (metric2), Industry2 (Metric1) ,..., 

In [20]:
def process_wide_format(input_file, output_file='mgdp1.csv'):  
    """  
    Process a CSV file to ensure it has proper "Industry (Metric)" column headers  
    and save it as a new CSV file for web interaction.  
    
    Parameters:  
    - input_file: Path to the input CSV file  
    - output_file: Path to save the processed CSV (default: 'mgdp1.csv')  
    """  
    try:  
        # Read the CSV file  
        df = pd.read_csv(input_file)  
        
        # Identify column names  
        columns = df.columns.tolist()  
        
        # Assuming first column is Date/Title  
        date_col = columns[0]  
        data_cols = columns[1:]  
        
        # Ensure proper column naming  
        new_columns = [date_col]  # Keep date/time column as is  
        
        for col in data_cols:  
            # Check if column already follows "Industry (Metric)" format  
            if re.match(r'^.*\(.*\).*$', col):  
                new_columns.append(col)  
            else:  
                # This handles columns that don't follow the format  
                # You may need to adjust this logic based on your actual data  
                new_columns.append(f"Unknown ({col})")  
        
        # Rename columns in the dataframe  
        df.columns = new_columns  
        
        # Convert date column to datetime if it contains dates  
        if df[date_col].dtype == 'object':  
            try:  
                df[date_col] = pd.to_datetime(df[date_col])  
                # Format date as YYYY-MM-DD  
                df[date_col] = df[date_col].dt.strftime('%Y-%m-%d')  
            except:  
                # If conversion fails, keep as is (might be a different type of identifier)  
                pass  
        
        # Save the processed dataframe  
        df.to_csv(output_file, index=False)  
        print(f"Successfully processed and saved to {output_file}")  
        
        # Display sample of the processed dataframe  
        print("\nSample of processed data:")  
        print(df.head())  
        
        # Display column information  
        print("\nColumn names in the processed file:")  
        for i, col in enumerate(df.columns):  
            print(f"{i}: {col}")  
            
        return df  
        
    except Exception as e:  
        print(f"Error processing file: {e}")  
        import traceback  
        traceback.print_exc()  
        return None  

# Usage  
input_file = "mgdp1.csv"  # Replace with your actual input file  
output_file = "mgdp2.csv"  
processed_df = process_wide_format(input_file, output_file)  

Successfully processed and saved to mgdp2.csv

Sample of processed data:
      Title Gross Value Added - Monthly (Index 1dp)  \
0      CDID                                    ECY2   
1  2016 JAN                                    91.2   
2  2016 FEB                                    91.4   
3  2016 MAR                                    91.6   
4  2016 APR                                    92.1   

  Agriculture, Forestry and Fishing (Index 1dp)  \
0                                          ECY3   
1                                          90.6   
2                                            88   
3                                          87.3   
4                                          87.5   

  Production Industries - Total (Index 1dp) Mining and Quarrying (Index 1dp)  \
0                                      ECY4                             ECY5   
1                                      99.3                            116.6   
2                                      99.1      

/tmp/ipykernel_5948/1614449950.py:39: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[date_col] = pd.to_datetime(df[date_col])
